In [17]:
import numpy as np

import drawSvg as draw

In [32]:
def written_number(n, z):
    if n:
        return str(n)[::-1]
    else:
        return z * '0'

In [211]:
phase, h_phase = -20, 10
handles = 5
quality = 2000

americaine = False
if americaine:
    # Americaine
    numbers = [0, 2, 14, 35, 23, 4, 16, 33, 21, 6, 18, 31, 19, 8, 12, 29, 25, 10, 27, 0, 1, 13, 36, 24, 3, 15, 34, 22, 5, 17, 32, 20, 7, 11, 30, 26, 9, 28]
else:
    # Française
    numbers = [0, 26, 3, 35, 12, 28, 7, 29, 18, 22, 9, 31, 14, 20, 1, 33, 16, 24, 5, 10, 23, 8, 30, 11, 36, 13, 27, 6, 34, 17, 25, 2, 21, 4, 19, 15, 32]

r_out, r_mid, r_in = 1, 0.85, 0.7
nb_width = 360 / len(numbers)
sw_ring = 0.0025
k_border = 'ivory'
k_outside = '#764929'

d = draw.Drawing(2.2, 2.2, origin='center', displayInline=False)

# Board
d.append(draw.Circle(0, 0, 1.05* r_out, fill=k_outside))

# Draw multiple circular arcs
angle = 90 + phase
red = True
ps = []
for n in numbers:
    if n:
        if red:
            stroke = '#BB2222'
        else:
            stroke = '#222222'
    else:
        stroke = '#007711'
    d.append(draw.Arc(0, 0, (r_out + r_in) / 2, angle - 1.01 * nb_width / 2, angle + 1.01 * nb_width / 2, stroke=stroke, stroke_width=r_out - r_in, fill_opacity=0))
    ps.append(draw.Arc(0, 0, (r_out + r_in) / 2, 180 + angle - nb_width / 2, 180 + angle + nb_width / 2, fill_opacity=0))
    angle += nb_width
    red ^= True

z = 0
for n in numbers:
    if not n:
        z += 1
    d.append(draw.Line(r_in * np.cos((angle - nb_width / 2) / 180 * np.pi), r_in * np.sin((angle - nb_width / 2) / 180 * np.pi), r_mid * np.cos((angle - nb_width / 2) / 180 * np.pi), r_mid * np.sin((angle - nb_width / 2) / 180 * np.pi), stroke=k_border, stroke_width=3 * sw_ring))
    d.append(draw.Text(written_number(n, z), 0.085, fill=k_border, path = ps.pop(0), center=True, lineOffset=-21))  # Text with font size 8
    angle += nb_width

angle = 90 + phase
for n in range(len(numbers) // 2 + 2):
    d.append(draw.Line(0, 0, r_in * np.cos(angle / 180 * np.pi), r_in * np.sin(angle / 180 * np.pi), stroke=k_border, stroke_width=2 * sw_ring))
    angle += 360 / (len(numbers) // 2 + 2)
        
# Outer ring
d.append(draw.Circle(0, 0, r_out - sw_ring / 3, stroke=k_border, stroke_width=2 * sw_ring, fill_opacity=0))
# Middle ring
d.append(draw.Circle(0, 0, r_mid - sw_ring / 2, stroke=k_border, stroke_width=5 * sw_ring, fill_opacity=0))
# Inner ring
d.append(draw.Circle(0, 0, r_in + sw_ring / 3, stroke=k_border, stroke_width=5 * sw_ring, fill_opacity=0))

# Lower Handle
d.append(draw.Circle(0, 0, 0.075 * r_out, fill=k_border))

l, w = 0.25, 0.04
pos = np.array([[0, l, l, 0], [-w / 3, -w / 2, w / 2, w / 3]])
rot = lambda theta: np.array([[np.cos(theta / 180 * np.pi), -np.sin(theta / 180 * np.pi)], [np.sin(theta / 180 * np.pi), np.cos(theta / 180 * np.pi)]])
theta = 90 + phase + h_phase
for h in range(handles):
    theta += 360 / handles
    rp = rot(theta) @ pos
    d.append(draw.Lines(rp[0, 0], rp[1, 0], rp[0, 1], rp[1, 1], rp[0, 2], rp[1, 2], rp[0, 3], rp[1, 3], close=True, fill=k_border, stroke=k_outside, stroke_width=sw_ring))

# Upper Handle
d.append(draw.Circle(0, 0, 0.05 * r_out, fill=k_border, stroke=k_outside, stroke_width=sw_ring))

d.setPixelScale(quality)  # Set number of pixels per geometry unit
# d.saveSvg('cover.svg')
# d.savePng('cover.png')

# Display in Jupyter notebook
d.rasterize()  # Display as PNG
d.savePng('cover.png')
d  # Display as SVG
